MDTS is not installing properly, but we can still just download the script and run them as is. First, let's get our files ready:

In [3]:
%%bash

cd ~/data/cnv/mdts

mkdir BAM
while read s; do
    ln -s /data/NCR_SBRB/simplex/BAM/${s}/${s}_sorted_RG_markduplicate_recalibrated.bam BAM/${s}.bam;
    ln -s /data/NCR_SBRB/simplex/BAM/${s}/${s}_sorted_RG_markduplicate_recalibrated.bai BAM/${s}.bai;
done < ../conifer84/samples.txt

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
%matplotlib inline

In [4]:
import glob
ped_files = ['/data/sudregp/multiplex_simplex/simplex.ped']
wes_prefix = ['CLIA', 'CCGO', 'WPS']
# fid = open('/home/sudregp/data/multiplex_simplex/samples_simplex_all.txt', 'r')
# exclude_list = [line.rstrip() for line in fid]
# fid.close()

# no controls/affected pair for comparison
exclude_list = ['CLIA_400165', 'CLIA_400164', 'CLIA_400155', 'CLIA_400146',
                'CLIA_400145', 'CLIA_400126', 'CLIA_400207', 'CLIA_400208',
                'CLIA_400209']
# missing one parent
exclude_list += ['CLIA_400169', 'CLIA_400168']
# family 9030
exclude_list += ['CCGO_800978', 'CCGO_800977', 'CCGO_800976', 'CCGO_800979',
                 'CCGO_800980', 'CLIA_400067']

trios = []
affected = []
controls = []
samples = []
famids = []
sexes = {}
for ped_file in ped_files:
    fid = open(ped_file, 'r')
    for line in fid:
        famid, sid, fa, mo, sex, aff = line.rstrip().split('\t')
        # if the current ID and its parents have WES data, and the sample is 
        # not in yet
        if (fa.split('_')[0] in wes_prefix and
            mo.split('_')[0] in wes_prefix and
            sid.split('_')[0] in wes_prefix and
            sid not in samples and
            (sid not in exclude_list or fa not in exclude_list or mo not in exclude_list)):
            fam = {}
            fam['child'] = sid
            if aff == '1':
                affected.append(sid)
            else:
                controls.append(sid)
            fam['father'] = fa
            fam['mother'] = mo
            fam['famid'] = famid
            sexes[sid] = sex
            trios.append(fam)
            samples += [sid, fa, mo]
            famids.append(famid)
    fid.close()
samples = set(samples)
famids = set(famids)
kids = set(affected + controls)
good_kids = kids

print 'Unique samples:', len(samples)
print 'Unique families:', len(famids)
print 'Unique children:', len(kids)

Unique samples: 84
Unique families: 19
Unique children: 46


In [6]:
fid = open('/data/sudregp/cnv/mdts/simplex84.ped', 'w')
fid.write('subj_id\tfamily_id\tfather_id\tmother_id\tgender\tbam_path\n')
for trio in trios:
    fid.write('\t'.join([trio['child'], trio['famid'], trio['father'],
                         trio['mother'], sexes[trio['child']],
                         'BAM/%s.bam' % trio['child']]) + '\n')
    fid.write('\t'.join([trio['father'], trio['famid'], '0', '0', '1',
                         'BAM/%s.bam' % trio['father']]) + '\n')
    fid.write('\t'.join([trio['mother'], trio['famid'], '0', '0', '2',
                         'BAM/%s.bam' % trio['mother']]) + '\n')
fid.close()

I'm running it in a R terminal to have a better look at the output, but here's a history of the commands (following from https://github.com/JMF47/MDTS/blob/master/vignettes/mdts.Rmd):

Can't tell yet if the chr1 file is enough, or if they need one per each chromosome. Also, need to check if this is the best reference file for us, and check if the other parameters are also appropriate.

Based on the output I'm seeing so far, I'd need one map_file for each chromossome. For example, the output of calcBins looks like this so far:

# TODO

*